In [1]:
pip install praw


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
import praw
from prawcore.exceptions import NotFound
import pandas as pd

arr=[]
reddit = praw.Reddit(
    client_id='i4zwfu6H7k0pmS_fL8V-fQ',
    client_secret='LBYNW08-w-sLW323oPGrOqL3tnV6QA',
    user_agent='python:sentiment_analysis:v1.0.0 (by u/NParab)'
    
)
i = 0
try:
    subreddit = reddit.subreddit('Technology')
    for post in subreddit.hot(limit=10000):
        print(i)
        if len(post.comments) > 0:
        #print(post.comments[0].body)
        #print(post.title)
        #print(post.selftext)
            arr.append([post.title,post.comments[0].body])
        i+=1
except NotFound:
    print("The requested resource was not found. Please check the subreddit name or resource ID.")
df_technology = pd.DataFrame(arr, columns=['Title', 'Comment'])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [24]:
df_technology

,Title,Comment
0,We’re a team of science & tech journalists cov...,Considering the large energy demand associated...
1,"JD Vance says Big Tech has ""too much power""",The call is coming from inside the house!
2,Trump accused of using AI to compose ‘slip sho...,whole country run by sociopathic trust fund ch...
3,"DeepSeek hit with large-scale cyberattack, say...","Begun, the AI Wars have."
4,DeepSeek releases new image model family,To clarify: This is NOT the model that has bee...
...,...,...
698,Microsoft and iFixit now sell official Xbox Se...,They still don't provide SSDs or tools to repl...
699,Truck with $1.8M in smuggled electronics seize...,This has to really byte for the people who got...
700,Driving blind: NYC subways steered by 1930s te...,I’m pretty sure their path is linear.
701,FBI forces Chinese malware to delete itself fr...,What’s with the comments on this post? Are the...


In [25]:
df_technology.to_csv('tech1.csv', index=False)
